# Machine learning toolbox Flux for julia

## Simple neural network tests using Flux.jl, copied or heavilly inspired by the [Flux.jl tutorial](https://fluxml.ai/Flux.jl/stable/models/quickstart/)

In [0]:
## Install Flux if needed
#import Pkg
#Pkg.add("Flux")

In [0]:
using Flux
using LinearAlgebra

##  XOR classification classic

In [0]:
# With Julia 1.7+, this will prompt if neccessary to install everything, including CUDA:
using Flux, Statistics

# Generate some data for the XOR problem: vectors of length 2, as columns of a matrix:
noisy = rand(Float32, 2, 1000)                                    # 2×1000 Matrix{Float32}
truth = map(col -> xor(col...), eachcol(noisy .> 0.5))            # 1000-element Vector{Bool}

# Define our model, a multi-layer perceptron with one hidden layer of size 3:
model = Chain(Dense(2 => 3, tanh), BatchNorm(3), Dense(3 => 2), softmax)

# The model encapsulates parameters, randomly initialised. Its initial output is:
out1 = model(noisy)                                               # 2×1000 Matrix{Float32}

# To train the model, we use batches of 64 samples:
mat = Flux.onehotbatch(truth, [true, false])                      # 2×1000 OneHotMatrix
data = Flux.DataLoader((noisy, mat), batchsize=64, shuffle=true);
first(data) .|> summary                                           # ("2×64 Matrix{Float32}", "2×64 Matrix{Bool}")

pars = Flux.params(model)  # contains references to arrays in model
opt = Flux.Adam(0.01)      # will store optimiser momentum, etc.

# Training loop, using the whole data set 1000 times:
for epoch in 1:1_000
    Flux.train!(pars, data, opt) do x, y
        # First argument of train! is a loss function, here defined by a `do` block.
        # This gets x and y, each a 2×64 Matrix, from data, and compares:
        Flux.crossentropy(model(x), y)
    end
end

pars  # has changed!
opt
out2 = model(noisy)

mean((out2[1,:] .> 0.5) .== truth)  # accuracy 94% so far!


In [0]:
using Plots  # to draw the above figure

p_true = scatter(noisy[1,:], noisy[2,:], zcolor=truth, title="True classification", legend=false,clims=(0,1))
p_raw =  scatter(noisy[1,:], noisy[2,:], zcolor=out1[1,:], title="Untrained network",legend=false, clims=(0,1))
p_done = scatter(noisy[1,:], noisy[2,:], zcolor=out2[1,:], title="Trained network", legend=true,clims=(0,1),label="")

plot(p_true, p_raw, p_done, layout=(1,3), size=(1000,330))


In [0]:
pars

In [0]:
model([0.25 0.75 0.25 0.75 ; 0.25 0.25 0.75 0.75]) #apply the trained model, with one point in each colunm

In [0]:
model # show summary

In [0]:
data = Flux.DataLoader((rand(2,10), rand(1,10)), batchsize=10, shuffle=false)
data